## Esta sección describe cómo las etiquetas duplicadas cambian el comportamiento de ciertas operaciones y cómo evitar que surjan duplicados durante las operaciones, o detectarlos si lo hacen.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df1 = pd.DataFrame([[0, 1, 2], [3, 4, 5]], columns=["A", "A", "B"])

In [3]:
df1

,A,A,B
0,0,1,2
1,3,4,5


In [4]:
# En este caso, tenemos etiquetas duplicadas en las columnas
# Si seleccionamos la columna B, no habrá problemas
df1['B']

0    2
1    5
Name: B, dtype: int64

In [5]:
# Pero si seleccionamos A, nos devuelve ambas columnas
df1['A']

,A,A
0,0,1
1,3,4


In [6]:
# lo mismo sucede, para filas que tienen como indice una letra o un texto
df2 = pd.DataFrame({"A": [0, 1, 2]}, index=["a", "a", "b"])

In [7]:
df2

,A
a,0
a,1
b,2


In [8]:
df2.loc['b', 'A']

2

In [9]:
df2.loc['a', 'A']

a    0
a    1
Name: A, dtype: int64

## Detectar de etiquetas duplicadas

Puede verificar si un índice (que almacena las etiquetas de fila o columna) es único con **Index.is_unique**. Esto devuelve un booleano.

In [10]:
df2

,A
a,0
a,1
b,2


In [12]:
df2.index.is_unique

False

In [13]:
# Podemos aplicar lo mismo para los indices de los columnas con: "columns.is_unique"
df2.columns.is_unique

True

#### Index.duplicated() devolverá un ndarray booleano que indica si una etiqueta se repite.

In [14]:
df2.index.duplicated()

array([False,  True, False])

#### Que se puede usar como un filtro booleano para eliminar filas duplicadas.

In [15]:
# Para hacerlo colocamos un tilde "~" delante del nombre de la tabla
df2.loc[~df2.index.duplicated(), :]

,A
a,0
b,2


## No permitir etiquetas duplicadas - allows_duplicate_labels=False

Como se indicó anteriormente, el manejo de duplicados es una característica importante al leer datos sin procesar. Dicho esto, es posible que desee evitar la introducción de duplicados como parte de una canalización de procesamiento de datos (desde métodos como pandas.concat(), rename(), etc.). Tanto Series como DataFrame no permiten etiquetas duplicadas llamando a **.set_flags(allows_duplicate_labels=False)**. (el valor predeterminado es permitirlos). Si hay etiquetas duplicadas, se generará una excepción.

In [16]:
pd.DataFrame([[0, 1, 2], [3, 4, 5]], columns=["A", "B", "C"],).set_flags(
    allows_duplicate_labels=False
)

,A,B,C
0,0,1,2
1,3,4,5


### allows_duplicate_labels=False / allows_duplicate_labels=True

#### Este atributo se puede marcar o configurar con allow_duplicate_labels, que indica si ese objeto puede tener etiquetas duplicadas.

In [17]:
df = pd.DataFrame({"A": [0, 1, 2, 3]}, index=["x", "y", "X", "Y"]).set_flags(
    allows_duplicate_labels=False
)

In [18]:
df

,A
x,0
y,1
X,2
Y,3


In [19]:
# Para comprobar
df.flags.allows_duplicate_labels

False

### DataFrame.set_flags() se puede usar para devolver un nuevo DataFrame con atributos como allow_duplicate_labels establecidos en algún valor

In [20]:
df2 = df.set_flags(allows_duplicate_labels=True)

In [21]:
df2

,A
x,0
y,1
X,2
Y,3


In [22]:
df2.flags.allows_duplicate_labels

True

#### Al ingresar datos sin procesar y desordenados, inicialmente puede leer los datos desordenados (que potencialmente tienen etiquetas duplicadas), desduplicar y luego rechazar los duplicados en el futuro, para asegurarse de que su canalización de datos no introduzca duplicados.

In [23]:
# raw = pd.read_csv("...")
# deduplicated = raw.groupby(level=0).first()  # remove duplicates
# deduplicated.flags.allows_duplicate_labels = False  # disallow going forward